### 9-6. 音の分類のために必要な Data の前処理をしてみよう
実世界の時系列Data である、音の分類問題を解いていく。

In [1]:
import librosa
import pandas as pd
import numpy as np
import IPython.display as ipd

In [8]:
# 学習Data の読み込み
train_data_dir = "./audio_dataset_3class/train/"
train_df = pd.read_csv("audio_dataset_3class./train.csv", index_col=0)

In [4]:
# Test-data の読み込み
test_data_dir = "./audio_dataset_3class/test/"
test_df = pd.read_csv("audio_dataset_3class/test.csv", index_col=0)

In [5]:
# 学習に使う Data の音声File名と Label の1部を表示する
train_df.head()

,label
fname,
969b4f60.wav,Cello
3e2bddda.wav,Cello
54bb57af.wav,Cello
9d59a719.wav,Applause
05f2c2a6.wav,Clarinet


音声File の名前と Label が対応づけられて格納されている。

In [6]:
# Label に使われている値とその数の一覧を表示
train_df["label"].value_counts()

Clarinet    130
Cello       125
Applause     61
Name: label, dtype: int64

Cello の音を読み込んで再生する

In [9]:
# Cello の音声Data のうちの１つを読み込む
data, rate = librosa.load(train_data_dir + train_df[train_df["label"] == "Cello"].index[0])

# 読み込んだ Data を再生する
ipd.Audio(data=data, rate=rate)

Cello の音声 Data の形を確認する

In [10]:
# 読み込んだ Cello の音声Data の形を確認
print(data.shape)
data

(113337,)


array([ 5.4858055e-04,  8.1656827e-04,  1.0076625e-03, ...,
       -2.8788178e-05, -2.3041464e-05, -1.3347699e-05], dtype=float32)

読み込んだ音の Data が入っている `data`変数の中身は、１次元の配列であることがわかる。

Data の前処理を実行する

In [11]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
sampling_rate = 8000
# 音の長さを3秒に切り取る
audio_duration = 3
audio_length = sampling_rate * audio_duration

In [13]:
# File名から音声Data を読み込む
def _load_files(data_dir, filenames):
    result = []
    for i, filename in enumerate(filenames):
        file_path = data_dir + filename
        data, _ = librosa.core.load(file_path, sr=sampling_rate, res_type="kaiser_fast")
        result.append(data)

    return result

In [14]:
def create_audio_dataset(train_df, test_df, train_data_dir, test_data_dir, label_dict):
    dim = (audio_length, 1)
    train_filenames = train_df.index
    test_filenames = test_df.index

    # 学習用Data と Test用Data の音声File を File名から読み込む
    _X_train = _load_files(train_data_dir, train_filenames)
    _X_test = _load_files(test_data_dir, test_filenames)

    # audio_length（ここでは３秒）に音の長さを揃える
    _X_train = pad_sequences(_X_train,
                             dtype="float32",
                             maxlen=audio_length,
                             padding="pre",
                             truncating="pre",
                             value=0.0).tolist()
    _X_test = pad_sequences(_X_test,
                            dtype="float32",
                            maxlen=audio_length,
                            padding="pre",
                            truncating="pre",
                            value=0.0).tolist()

    # 音の Data を StandardScaler で平均０、分散を１に補正する
    scaler = StandardScaler()
    scaler = scaler.fit(_X_train + _X_test)
    _X_train = scaler.transform(_X_train)
    _X_test = scaler.transform(_X_test)

    X_train = np.empty((len(train_filenames), *dim))
    for index, data in enumerate(_X_train):
        X_train[index,] = [[d] for d in data]

    X_test = np.empty((len(test_filenames), *dim))
    for index, data in enumerate(_X_test):
        X_test[index,] = [[d] for d in data]

    # 以下からは label の作成
    labels_train = train_df["label"]
    labels_test = test_df["label"]

    y_train = np.empty(len(labels_train), dtype=int)
    for i, label in enumerate(labels_train):
        y_train[i] = label_dict[label]

    y_test = np.empty(len(labels_test), dtype=int)
    for i, label in enumerate(labels_test):
        y_test[i] = label_dict[label]

    # one-hot encoding する
    Y_train = to_categorical(y_train, num_classes=len(label_dict))
    Y_test = to_categorical(y_test, num_classes=len(label_dict))

    return X_train, Y_train, X_test, Y_test

In [15]:
audio_label_dict = {"Cello": 0, "Clarinet": 1, "Applause": 2}
X_train, Y_train, X_test, Y_test = create_audio_dataset(train_df, test_df, train_data_dir, test_data_dir, audio_label_dict)